In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Libraries and Imports

In [2]:
%%capture
!pip install -q llama-index llama-index-llms-groq llama-index-embeddings-fastembed llama-index-vector-stores-kdbai llama-index-callbacks-opik llama-index-postprocessor-colbert-rerank
!pip install -q groq kdbai-client opik litellm
!pip install -q pandas torch transformers
!pip install -q tqdm nest_asyncio
!pip install -q qdrant-client llama-index-vector-stores-qdrant
!pip install -q pip install llama-index-retrievers-bm25

In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    StorageContext,
    Settings,
    Document,
    PromptTemplate,
    set_global_handler)
from llama_index.core.node_parser import (
    SentenceSplitter,
    MarkdownNodeParser)
from llama_index.core.llms import ChatMessage
from llama_index.llms.groq import Groq
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core.indices import VectorStoreIndex, SimpleKeywordTableIndex, KeywordTableIndex
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import (
    RouterQueryEngine,
    SubQuestionQueryEngine,
    TransformQueryEngine)
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.selectors import LLMMultiSelector, PydanticMultiSelector



import groq
import kdbai_client as kdbai

import litellm
from litellm.integrations.opik.opik import OpikLogger
from opik import track
from opik.opik_context import get_current_span_data
from opik import Opik

import pandas as pd
from typing import List, Dict
import matplotlib.pyplot as plt
from llama_index.core.response.notebook_utils import display_response

import time
from tqdm.notebook import tqdm
from IPython.display import Markdown, display

import os
import nest_asyncio
nest_asyncio.apply()

# Configure Comet Opik

Traces can be observed on Comet's Opik after executing the next cell

https://docs.llamaindex.ai/en/stable/examples/observability/OpikCallback/

In [4]:
#Add the LiteLLM OpikTracker to log traces and steps to Opik:
os.environ["OPIK_PROJECT_NAME"] = "Omdena_SL_RAG_Agents_v2.1"
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

In [5]:
# # Configure Opik from google secrets
from google.colab import userdata
os.environ["OPIK_API_KEY"]  = userdata.get('OPIK_API_KEY')
os.environ["OPIK_WORKSPACE"] = userdata.get('OPIK_WORKSPACE')

In [6]:
# setting Opik API Key
#os.environ["OPIK_API_KEY"] = (
#    os.environ["OPIK_API_KEY"]
#    if "OPIK_API_KEY" in os.environ
#    else getpass("OPIK API Key: ")
#)

#os.environ["OPIK_WORKSPACE"] = (
#    os.environ["OPIK_WORKSPACE"]
#    if "OPIK_WORKSPACE" in os.environ
#    else getpass("OPIK_WORKSPACE Key: ")
#)

In [7]:
set_global_handler("opik")

# Data Loading

In [8]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena_SL/data/v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [9]:
set([doc.metadata['class'] for doc in all_documents])

{'circular', 'guideline', 'regulatory'}

In [10]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [11]:
guideline_docs = [doc for doc in all_documents if doc.metadata['class'] == 'guideline']
len(guideline_docs)

13

In [12]:
regulatory_docs = [doc for doc in all_documents if doc.metadata['class'] == 'regulatory']
len(regulatory_docs)

47

# Create KDB.AI session and table¶

# Embedding Model

In [13]:
def setup_embedding_model():
    """
    Setup embedding model
    """
    model_name = 'BAAI/bge-small-en-v1.5'
    return FastEmbedEmbedding(
        model_name=model_name
        )

embed_model = setup_embedding_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

# Groq + KDBAI API Setup

In [14]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')

In [15]:
def setup_groq_llm():
    """
    Setup Groq LLM
    """
    groq_api_key = GROQ_API_KEY
    if not groq_api_key:
        raise ValueError("Please set GROQ_API_KEY environment variable")

    return Groq(
        api_key=groq_api_key,
        model="llama3-groq-8b-8192-tool-use-preview",
        temperature=0.0
    )

llm = setup_groq_llm()


# KDBAI API + Session Setup

In [16]:
KDBAI_API_KEY = userdata.get('KDBAI_API_KEY')
KDBAI_SESSION_ENDPOINT = userdata.get('KDBAI_ENDPOINT')

In [17]:
def setup_kdbai_api():
  """
  Setup KDBAI Session Endpoint and API
  """

  kdbai_endpoint = KDBAI_SESSION_ENDPOINT
  if not kdbai_endpoint:
        raise ValueError("Please set KDBAI_SESSION_ENDPOINT environment variable")

  kdbai_api_key = KDBAI_API_KEY
  if not kdbai_api_key:
        raise ValueError("Please set KDBAI_API_KEY environment variable")

  return kdbai.Session(
    endpoint=f"{kdbai_endpoint}",
    api_key=f"{kdbai_api_key}"
    )

session = setup_kdbai_api()

# KDBAI Vector Store Setup

## Session Database

In [18]:
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_rag_database_2",
 KDBAI database "srilanka_tea"]

In [19]:
# ensure no database called "srilanka_tea" exists
try:
    session.database("srilanka_rag_database").drop()
except kdbai.KDBAIException:
    pass

# Create the database
db = session.create_database("srilanka_rag_database")
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_rag_database_2",
 KDBAI database "srilanka_tea"]

## Table Schema + Creation

In [20]:
# List all of the tables in the db
db.tables

[]

In [21]:
# Table - name & schema
table_name = "omdena_sl_rag"

# Same table schema and table index for each class
table_schema = [
        dict(name="document_id", type="bytes"),
        dict(name="text", type="bytes"),
        dict(name="embeddings", type="float32s"),
        dict(name="issue_date", type="str"),
        dict(name="issuing_authority", type="str"),
        dict(name="class", type="str"),
        dict(name="reference_number", type="str"),
        dict(name="title", type="str"),
        dict(name="original_doc_id", type="int16"),
    ]

indexFlat = {
        "name": "flat_index",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 384, 'metric': 'CS'} # For similarity metric, choose from Euclidean Distance (L2), Dot Product (IP), or Cosine Similarity (CS).
    }

In [22]:
# First ensure the tables does not already exist
try:
    db.table(table_name).drop()
except kdbai.KDBAIException:
    pass

# Create tables
circular_table = db.create_table(table_name, table_schema, indexes=[indexFlat])

db.tables

[KDBAI table "omdena_sl_rag"]

In [23]:
circular_table.indexes

[{'name': 'flat_index',
  'type': 'flat',
  'column': 'embeddings',
  'params': {'metric': 'CS', 'dims': 384}}]

# Data Loading

In [24]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena_SL/data/v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [25]:
set([doc.metadata['class'] for doc in all_documents])

{'circular', 'guideline', 'regulatory'}

In [26]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [27]:
guideline_docs = [doc for doc in all_documents if doc.metadata['class'] == 'guideline']
len(guideline_docs)

13

In [28]:
regulatory_docs = [doc for doc in all_documents if doc.metadata['class'] == 'regulatory']
len(regulatory_docs)

47

## Insert Data into Tables

In [29]:
Settings.llm = llm
Settings.embed_model = embed_model

In [30]:
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=16)

nodes = splitter.get_nodes_from_documents(all_documents)

In [31]:
# uses SimpleVectorStore, SimpleDocumentStore, SimpleIndexStore to store embeddings, nodes and extra metadata respectively
storage_context = StorageContext.from_defaults()

storage_context.docstore.add_documents(nodes)

In [32]:
# uses regex to extract keywords
regex_keyword_index = SimpleKeywordTableIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)

# Summary Query Tool
summary_index = SummaryIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,)


# uses SimpleVectorStore to store embeddings
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)




Extracting keywords from nodes:   0%|          | 0/781 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/781 [00:00<?, ?it/s]

OPIK: Started logging traces to the "Omdena_SL_RAG_Agents_v2.1" project at https://www.comet.com/opik/sherilyn-396/redirect/projects?name=Omdena_SL_RAG_Agents_v2.1.


In [33]:
from llama_index.core import PromptTemplate

# creating a custom prompt
QA_PROMPT_TMPL = """
Context:
---------------------
{context_str}
---------------------
Based on the context above, answer the question below. If the answer is not in the context,
inform the user without making up an answer. Additionally, provide a relevance score for the answer.

Question: {query_str}
Answer (with relevance score):
"""
QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

#Uses Colbert V2 model as a reranker to rerank documents according to the fine-grained similarity between query tokens and passage tokens. Returns the top N ranked nodes.
colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

# Uses a cross encoder model as a reranker to rerank documents. Returns the top N ranked nodes.
crossencoder_reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)


# regex keyword
keyword_query_engine = regex_keyword_index.as_query_engine(
    text_qa_template=QA_PROMPT,
    node_postprocessors=[crossencoder_reranker]
)


# summary
summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
        node_postprocessors=[crossencoder_reranker]
    )

# only vector semantic
vector_query_engine = vector_index.as_query_engine(text_qa_template=QA_PROMPT,
                                                   node_postprocessors=[crossencoder_reranker]
                                                   )

# Vector and bm25
## When querying with hybrid mode, we can set similarity_top_k and sparse_top_k separately.
##sparse_top_k represents how many nodes will be retrieved from each dense and sparse query.
##For example, if sparse_top_k=5 is set, that means I will retrieve 5 nodes using sparse vectors and 5 nodes using dense vectors.
##similarity_top_k controls the final number of returned nodes.
##In the above setting, we end up with 10 nodes.
## A fusion algorithm is applied to rank and order the nodes from different vector spaces (relative score fusion in this case).
## similarity_top_k=2 means the top two nodes after fusion are returned.

hybrid_query_engine = vector_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=5, sparse_top_k=10, text_qa_template=QA_PROMPT
)



# HYDE
hyde = HyDEQueryTransform(
    include_original=True,
    )

hyde_query_engine = TransformQueryEngine(
    query_engine = vector_index.as_query_engine(),
    query_transform = hyde
    )



tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/62.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [34]:
response = vector_query_engine.query(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)

In [35]:
print(response)

Exobasidium vexans is a fungus that causes blister blight disease in tea. It spreads through wind-borne spores that require high relative humidity (85%) for spore production and infection. The spores are killed by direct sunlight or desiccation, and the disease is most prevalent in conditions of persistent mist and dew. (Relevance score: 95)


In [36]:
response = keyword_query_engine.query(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)

In [37]:
print(response)

I'm sorry but I do not have the capability to perform this task for you, I am happy to help you with any other queries you may have.


### QueryEngineTool


In [64]:
from llama_index.core.tools import QueryEngineTool


# keyword tool
keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    name="keyword_engine",
    description="Useful when the question is not fully formed and consistes of keywords or topics.",
)


# summarization and listing tool
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    name="summary_engine",
    description="Useful when the user asks to summarize or list down points about a topic",
)


# semantic vector tool
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="vector_engine",
    description="Useful when the user asks specific context type fully-formed question.",
)

# vector and bm25
hybrid_tool = QueryEngineTool.from_defaults(
    query_engine=hybrid_query_engine,
    name="hybrid_engine",
    description="Useful when the user asks a fully-formed question.",
)


#Hyde
hyde_tool = QueryEngineTool(
    query_engine=hyde_query_engine,
    metadata=ToolMetadata(
        name="hyde_engine",
        description="Useful for answering complex queries",
    ),
)

# subquery
# Initialize SubQuestionQueryEngine
sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[keyword_tool, summary_tool, vector_tool]
)

# Wrap SubQuestionQueryEngine in a QueryEngineTool
sub_question_tool = QueryEngineTool(
    query_engine=sub_question_query_engine,
    metadata=ToolMetadata(
        name="sub_question_engine",
        description="Useful when multiple queries are present in a question so breaking them into sub-questions",
    ),
)




## Define a Router Query Engine and TREE_SUMMARIZE_PROMPT_TMPL



In [39]:

TREE_SUMMARIZE_PROMPT_TMPL = """
Context from multiple sources is below. Each source may have a relevance score.

---------------------
{context_str}
---------------------

Based on the information from the sources above, answer the question below.

If the answer is not in the context, inform the user without making up an answer.

Question: {query_str}
Answer:
"""

tree_summarize = TreeSummarize(
    summary_template=PromptTemplate(TREE_SUMMARIZE_PROMPT_TMPL)
)

In [68]:

# Using the RouterQueryEngine and PydanticMultiselector to select from the various tool options provided and in a structured output format

query_engine = RouterQueryEngine(
    #selector=LLMMultiSelector.from_defaults(),
    selector = PydanticMultiSelector.from_defaults(),
    query_engine_tools=[
        keyword_tool,
        summary_tool,
        vector_tool,
        #hybrid_tool,
        #hyde_tool,
        #sub_question_tool
    ],
    summarizer=tree_summarize,
    verbose=True,
)

In [41]:
response = await query_engine.aquery(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)
print(response)

Selecting query engine 0: Useful for answering finding documents based on keywords and incomplete questions from a user..
Selecting query engine 1: Useful for summarization questions..
Selecting query engine 2: Useful for answering specific context type from a user..
Exobasidium vexans is a fungus that causes blister blight disease in tea. It spreads through wind-borne spores that require high relative humidity (85%) for spore production and infection.


In [42]:
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

**`Final Response:`** Exobasidium vexans is a fungus that causes blister blight disease in tea. It spreads through wind-borne spores that require high relative humidity (85%) for spore production and infection.

---

**`Source Node 1/8`**

**Node ID:** e0430337-b77c-4981-9cab-e82983710e8d<br>**Similarity:** 1.0888593196868896<br>**Text:** tea field, factory and office, while complying with the Government directives and Ministry of Health and Indigenous Medical Services guidelines issued time to time.

The staff and workers of following categories should not report for work;

1. Those having fever, with or without acute onset respiratory symptoms such as cough, runny nose, sore throat and/or shortness of breath.
2. Those who have had contact with suspected or confirmed case of COVID-19 for the last 14 days
3. Those who are quar...<br>**Metadata:** {'original_doc_id': 92, 'class': 'guideline', 'issuing_authority': 'Tea Research Institute', 'title': '# GUIDELINES ON PREVENTION AND CONTROL OF SPREAD OF COVID-19 IN TEA PLANTATIONS', 'issue_date': 'May 2020', 'reference_number': 'Guideline No: 02/2020'}<br>

---

...

**`Source Node 7/8`**

**Node ID:** a343a875-19f4-43bc-ae1d-1efe40f4f0bb<br>**Similarity:** 1.210833191871643<br>**Text:** # ADVISORY CIRCULAR

# No.DM JHL 925VynvT

Issued in: February 2024

Serial No. 04/24

# PROTECTION OF TEA FROM BLISTER BLIGHT

This Advisory Circular supersedes the Advisory Circular DM 1, Serial No. 1/02 issued in October 2002 and related previous Advisory Circulars and links with Advisory Circular PU 2.

# 1. Introduction

Blister blight disease of tea is caused by a fungus (*Exobasidium vexans* Massee) and is spread by wind-borne spores. High relative humidity (85%) is required for spore ...<br>**Metadata:** {'original_doc_id': 33, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'Protection of Tea from Blister Blight', 'issue_date': 'February 2024', 'reference_number': 'No. DM JHL 925VynvT, Serial No. 04/24'}<br>

---

**`Source Node 8/8`**

**Node ID:** 72788fb4-faff-4830-8879-818c7cecd0fd<br>**Similarity:** -5.92352294921875<br>**Text:** # ADVISORY CIRCULAR No.DM 5

Issued in: February 2024

Serial No. 08/24

# PROTECTION OF TEA FROM STEM AND BRANCH CANKER DISEASE

This Advisory Circular supersedes the Advisory Circular DM 5 Serial No 20/03 issued in September 2003 and related previous Advisory Circulars and links with Advisory Circular PU 2.

# 1. Introduction

Stem and branch canker caused by fungi of family Botryosphaeriaceae, Lasiodiplodia theobromae, Botryosphaeria mamane & Botryosphaeria dothidea (generally known as Mac...<br>**Metadata:** {'original_doc_id': 36, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'PROTECTION OF TEA FROM STEM AND BRANCH CANKER DISEASE', 'issue_date': 'February 2024', 'reference_number': 'No.DM 5 / Serial No. 08/24'}<br>

In [43]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='e0430337-b77c-4981-9cab-e82983710e8d', embedding=None, metadata={'original_doc_id': 92, 'class': 'guideline', 'issuing_authority': 'Tea Research Institute', 'title': '# GUIDELINES ON PREVENTION AND CONTROL OF SPREAD OF COVID-19 IN TEA PLANTATIONS', 'issue_date': 'May 2020', 'reference_number': 'Guideline No: 02/2020'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='b3b9fb14-44ca-4d5d-bd0e-8c2ad0a67778', node_type='4', metadata={'original_doc_id': 92, 'class': 'guideline', 'issuing_authority': 'Tea Research Institute', 'title': '# GUIDELINES ON PREVENTION AND CONTROL OF SPREAD OF COVID-19 IN TEA PLANTATIONS', 'issue_date': 'May 2020', 'reference_number': 'Guideline No: 02/2020'}, hash='430e3875ef691165da02e911847c4bc86876a03da418c9bfa158c394f9c515e8'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='21ff971e-9f32-4c24-9125-8b406d8f3904', node_type='1', me

In [44]:
qa_dict = {}

In [45]:
%%time

input_query = "What are the basic requirements needed to be fulfilled to be a Tea Exporter?"
result = query_engine.query(input_query)
print("result:",result)
qa_dict[input_query] = result

display_response(
    result, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: Useful for answering finding documents based on keywords and incomplete questions from a user..
Selecting query engine 1: Useful for summarization questions..
Selecting query engine 2: Useful for answering specific context type from a user..
result: To be a Tea Exporter, one needs to have an adequate organization, sufficient financial resources, and suitable premises for engaging in the business of an exporter of tea. Additionally, the person should hold a valid business registration certificate, have a minimum capital of Rs 4.0 Million, and ensure the premises are suitable for the tea export business.


**`Final Response:`** To be a Tea Exporter, one needs to have an adequate organization, sufficient financial resources, and suitable premises for engaging in the business of an exporter of tea. Additionally, the person should hold a valid business registration certificate, have a minimum capital of Rs 4.0 Million, and ensure the premises are suitable for the tea export business.

---

**`Source Node 1/8`**

**Node ID:** 2ac37e40-adb8-4181-8ce2-a98f1fd51d81<br>**Similarity:** -0.00860961526632309<br>**Text:** If the Government Analysts Department confirms that any teas prepared for export by any registered exporter of tea are below the Minimum Quality Standard and/or are contaminated, such analytical costs will be recovered from the exporter concerned.

If the Government Analysts Department confirms that any teas which have been stored by registered exporters of tea intending to utilize them for export purposes or otherwise, are below the Minimum Quality Standard and/or are contaminated, such anal...<br>**Metadata:** {'original_doc_id': 168, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'SRILANKA TEABOARD AT THE TEA EXPORT SECTION OF PROCEDURES RELATED TO EXPORT OF TEA 16 of 1959', 'issue_date': '20th December 2001', 'reference_number': 'nan'}<br>

...

**Node ID:** 03b098b3-2a10-48e3-bc10-57126e9a754d<br>**Similarity:** 1.9556758403778076<br>**Text:** # SRI LANKA TEA BOARD

# CExOM TEA

Ref: TC/EIRP 015' November 2010.

# Guidelines to New Applicants [Registered Exporters]

The Sri Lanka Tea Board has set criteria for registration as an exporter of tea in terms of the Tax and renewal of registration. The minimum requirements for registration as exporters of tea are stated below:

# (I) Business Registration

Applicants should hold valid business registration certificates issued by the Provincial Registrar of Business Names in the case of D...<br>**Metadata:** {'original_doc_id': 166, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Guidelines to New Applicants [Registered Exporters]', 'issue_date': 'November 2010', 'reference_number': 'TC/EIRP 015'}<br>

---

**`Source Node 8/8`**

**Node ID:** 89b1b349-f552-42e4-aed8-407802b50687<br>**Similarity:** -1.1405876874923706<br>**Text:** A residence or portion of a residence will not be allowed for storing.

# (iv) Staff

Taster in their staff who should possess trading experience in a recognized tea. Applicants for registration as exporters of tea should have at least one tea experience and ability acquired will be verified by the Board. The names of exporting/brokering firm for a minimum period of 6 months should be indicated in the application. Inspection and interview will be conducted by the panel appointed by the Sri La...<br>**Metadata:** {'original_doc_id': 166, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Guidelines to New Applicants [Registered Exporters]', 'issue_date': 'November 2010', 'reference_number': 'TC/EIRP 015'}<br>

CPU times: user 1min 19s, sys: 5.78 s, total: 1min 25s
Wall time: 1min 26s


In [69]:
%%time

input_query = "In terms of tea farming, what is medium shade?"
result = query_engine.query(input_query)
qa_dict[input_query] = result

print(reults)

display_response(
    result, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: The question is not fully formed and consists of keywords or topics..
Selecting query engine 1: The user asks to summarize or list down points about a topic..


**`Final Response:`** Medium shade in tea farming refers to the level of shading provided by trees planted at an initial distance of 3.0 m x 3.6 m, which is later thinned down to 6.0 m x 7.2 m after three years. This type of shading is suitable for tea plants and promotes healthy growth.

---

**`Source Node 1/6`**

**Node ID:** b1b03e9a-85e4-4665-94a9-d2eee7e8ab48<br>**Similarity:** -0.9368438720703125<br>**Text:** # Establishment of Medium Shade

# 3.1

Calliandra calothrysus thrives well in elevations between 300 and 1500 m. Calliandra should be established in the following manner: It, however, thrives best at 1300 m.
To establish Calliandra directly in new clearings, 3 to 4 seeds should be sown at one place, on soil mound, where the planting hole is cut to a depth of 45 cm: After the seeds germinate, the seedlings should be thinned down, leaving one vigorous plant to be planted out in the new clearin...<br>**Metadata:** {'original_doc_id': 70, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'ADVVISORY CIRCULAR No.SI 925 SHADING IN TEA', 'issue_date': 'March 2003', 'reference_number': 'SI 925'}<br>

...

**Node ID:** b86311ca-ab45-4a7a-8123-90213c45e3af<br>**Similarity:** -1.7052558660507202<br>**Text:** 0 m x 7.2 m (20 ft x 24 ft)|
|Medium shade|Acacia decunens suitable above 1370 m (4500 ft)|3.0 m x 3.6 m (10 ft x 12 ft)|6.0 m x 7.2 m (20 ft x 24 ft)|

# (b) Mid-country Wet Zone: 600 - 1200 m (2000 - 4000 ft)

|Type of shade|Shade tree species|Initial stand|Final stand|
|---|---|---|---|
|High shade|Grevillea robusta|6.0 m x 6.0 m (20 ft x 20 ft)|12.0 m x 12.0 m (40 ft x 40 ft)|
|High shade|Albizzia moluccana|6.0 m x 6.0 m (20 ft x 20 ft)|12.0 m x 12.0 m (40 ft x 40 ft)|
|High shade|Albizzi...<br>**Metadata:** {'original_doc_id': 70, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'ADVVISORY CIRCULAR No.SI 925 SHADING IN TEA', 'issue_date': 'March 2003', 'reference_number': 'SI 925'}<br>

---

**`Source Node 6/6`**

**Node ID:** 75c68207-fd86-4079-bd18-1c52f2dafeba<br>**Similarity:** -1.9840571880340576<br>**Text:** Felling of shade trees should be done after all the leaves have turned yellow and fallen. It is preferable not to allow the stumps to remain above the soil and any such stumps remaining should be covered with earth after cutting below the soil level.

New shade trees should be planted well before the old ones are removed, so that the field is not left without shade. Ring-barking is not necessary if the roots can be completely removed.

# Replacement of Shade trees

Generally, in about 10 year...<br>**Metadata:** {'original_doc_id': 70, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'ADVVISORY CIRCULAR No.SI 925 SHADING IN TEA', 'issue_date': 'March 2003', 'reference_number': 'SI 925'}<br>

CPU times: user 1min 21s, sys: 8.8 s, total: 1min 29s
Wall time: 1min 30s


In [53]:
%%time

input_query = "Who can be a Green leaf dealer?"
result = query_engine.query(input_query)
qa_dict[input_query] = result

display_response(
    result, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: Can find documents based on keywords and incomplete questions related to Green leaf dealers..
Selecting query engine 1: Can summarize information about Green leaf dealers..


**`Final Response:`** According to the Tea Control Act, No. 51 of 1957, a licensed dealer in green tea leaf can be a person or a firm. This includes individuals and firms carrying on business in partnership.

---

**`Source Node 1/6`**

**Node ID:** 694c6382-1570-4005-9c5a-75f84189246b<br>**Similarity:** 2.3120715618133545<br>**Text:** (8) Every dealer’s licence shall-

- (a) be in the prescribed form;
- (b) state the name and address of the licensee, or if such licence is in respect of a firm, the names and addresses of the partners of that firm;
- (c) state whether the licence is in respect of made tea, green tea leaf or both those commodities;
- (d) describe the premises (hereafter in this Act referred to as the ‘licensed premises’) at which the business of a dealer is authorized to be carried on; and
- (e) state the per...<br>**Metadata:** {'original_doc_id': 19, 'class': 'regulatory', 'issuing_authority': 'Parliament of Sri Lanka', 'title': 'Tea Control Act, No. 51 of 1957', 'issue_date': 'nan', 'reference_number': '51 of 1957'}<br>

...

**Node ID:** 17c572a3-0945-4ce4-acbc-f3ce6a243c48<br>**Similarity:** 2.1638731956481934<br>**Text:** # Sale and purchase of tea.

# 16.

(1) No person other than a licensed dealer in made tea or a registered manufacturer, or a person acting on behalf of such licensed dealer or registered manufacturer, shall sell or deliver any quantity of made tea exceeding the prescribed quantity to any other person:

Provided, however, that nothing in the preceding provisions of this subsection shall be deemed to prohibit the registered proprietor of a registered estate or small holding from selling or del...<br>**Metadata:** {'original_doc_id': 19, 'class': 'regulatory', 'issuing_authority': 'Parliament of Sri Lanka', 'title': 'Tea Control Act, No. 51 of 1957', 'issue_date': 'nan', 'reference_number': '51 of 1957'}<br>

---

**`Source Node 6/6`**

**Node ID:** 92941313-0bc6-4218-b9cf-ec5307a8d139<br>**Similarity:** 1.8691749572753906<br>**Text:** # Discrepancy in quantity of made tea or green tea leaf in licensed premises of licensed dealer or in tea factory of manufacturer and quantity entered in the register.

43.

1. (a) Where there is a discrepancy between the quantity of made tea or green tea leaf found in the licensed premises of a licensed dealer and the quantity thereof which according to the entries in the register maintained by him under subsection (2) of section 18 should be in such premises, or
2. (b) where there is a disc...<br>**Metadata:** {'original_doc_id': 19, 'class': 'regulatory', 'issuing_authority': 'Parliament of Sri Lanka', 'title': 'Tea Control Act, No. 51 of 1957', 'issue_date': 'nan', 'reference_number': '51 of 1957'}<br>

CPU times: user 1min 22s, sys: 9.56 s, total: 1min 31s
Wall time: 1min 36s


In [54]:
%%time

input_query = "What is the annual registration fee for the Tea factory?"
result = query_engine.query(input_query)
qa_dict[input_query] = result

display_response(
    result, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: Useful for answering finding documents based on keywords and incomplete questions from a user..
Selecting query engine 1: Useful for summarization questions..
Selecting query engine 2: Useful for answering specific context type from a user..


**`Final Response:`** The annual registration fee for a Tea factory in the Large Category (Over 1 Million kg per annum) is Rs. 25,000/-

---

**`Source Node 1/8`**

**Node ID:** 89b1b349-f552-42e4-aed8-407802b50687<br>**Similarity:** 0.11234887689352036<br>**Text:** A residence or portion of a residence will not be allowed for storing.

# (iv) Staff

Taster in their staff who should possess trading experience in a recognized tea. Applicants for registration as exporters of tea should have at least one tea experience and ability acquired will be verified by the Board. The names of exporting/brokering firm for a minimum period of 6 months should be indicated in the application. Inspection and interview will be conducted by the panel appointed by the Sri La...<br>**Metadata:** {'original_doc_id': 166, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Guidelines to New Applicants [Registered Exporters]', 'issue_date': 'November 2010', 'reference_number': 'TC/EIRP 015'}<br>

...

**Node ID:** 3494b506-7d52-429b-85d8-119137c2f0ce<br>**Similarity:** 0.35729339718818665<br>**Text:** 500/-|
|Annual Statistical Bulletin|~do-|1,000/-|2,500/-|
|Monthly Statistical Summary|~do-|~No-|500/-|
|Statistical Reports|~do-|70/= per page|150/- per page|
|Annual Report|~do-|~No-|750/-|
|Issue of Quality Certificate|~do-|250/-|2,500/-|
|Tasting of tea sample by the Panel|~do-|~No-|500/- per sample|
|Lion Logo Registration|~do-|REG -4,500/- for 3 years|Registration-5,000/- per annum Renewal-5,000/-|
|Ozone Friendly Logo Registration|~do-|No|Registration-5,000/- Renewal-5,000/-|
|GI Regis...<br>**Metadata:** {'original_doc_id': 164, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': "LEVY OF FEES & CHARGES FOR BOARD'S SERVICES", 'issue_date': '01st September 2010', 'reference_number': 'OR/1/87'}<br>

---

**`Source Node 8/8`**

**Node ID:** 6f05d91c-3c65-4dec-90b6-1dbdd32566d5<br>**Similarity:** -2.556396007537842<br>**Text:** Acting Statistician
20. Audit Superintendent; Govt. Audit Unit;
# SRI LANKA TEA BOARD

|TYPE OF FEE LEVIED|Registration Fee (REGISTRATION & RENEWAL FEE)|Charges|
|---|---|---|
|Exporter/Renewal of Registration of Tea|REG - 10,000/- Renewal - 25,000/- Renewal-L - 250,000/- Renewal-M - 125,000/- Renewal-S - 50,000/-|Please Note VAT and NBT Applicable|
|Importer/Renewal of Registration|REG - 1,000/- Renewal - 5,000/-| |
|Packer/Renewal of Registration|REG - 1,000/- Renewal - 10,000/-| |
|Warehou...<br>**Metadata:** {'original_doc_id': 164, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': "LEVY OF FEES & CHARGES FOR BOARD'S SERVICES", 'issue_date': '01st September 2010', 'reference_number': 'OR/1/87'}<br>

CPU times: user 1min 21s, sys: 8.59 s, total: 1min 29s
Wall time: 1min 30s


In [55]:
%%time

input_query = "Under which clause of the act provides the Tea Commissioner to Reasonable price?"
result = query_engine.query(input_query)
qa_dict[input_query] = result

display_response(
    result, show_source=True, source_length=500, show_source_metadata=True
)

Selecting query engine 0: The Tea Commissioner can use this to find documents based on keywords and incomplete questions from a user..
Selecting query engine 1: This clause is relevant for summarization questions..


**`Final Response:`** The Tea Commissioner determines the reasonable price for green leaf based on the current prices at a Colombo tea auction under section 5 of the Tea (Tax and Control of Export) Act, No. 16 of 1959.

---

**`Source Node 1/6`**

**Node ID:** d754d344-c52b-4818-b7c7-f08e07498079<br>**Similarity:** 3.403359889984131<br>**Text:** My Ref: NF / BL/146

Commissioner 's Tea Sri Lanka

Board

Tea

P. O. Division Box 363

Colombo 01

23rd March 1999

TQ: ALL REGISTERED TEA MANUFACTURERS

Dear Sir,

# REASONABLE PRICE FOR GREEN LEAF

Reference: 28th June, 1994, requested on the above subject Circular No. MF / BL 136

It has been decided to amend the price of green leaf from 22.227% to 21.5% for out-turn now.

This will come into effect from: 01st March, 1999 under the reasonable formula.

Please take action accordingly.

You...<br>**Metadata:** {'original_doc_id': 121, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'REASONABLE PRICE FOR GREEN LEAF', 'issue_date': '23rd March 1999', 'reference_number': '28th June, 1994 (Circular No. MF / BL 136)'}<br>

...

**Node ID:** d754d344-c52b-4818-b7c7-f08e07498079<br>**Similarity:** 3.4033589363098145<br>**Text:** My Ref: NF / BL/146

Commissioner 's Tea Sri Lanka

Board

Tea

P. O. Division Box 363

Colombo 01

23rd March 1999

TQ: ALL REGISTERED TEA MANUFACTURERS

Dear Sir,

# REASONABLE PRICE FOR GREEN LEAF

Reference: 28th June, 1994, requested on the above subject Circular No. MF / BL 136

It has been decided to amend the price of green leaf from 22.227% to 21.5% for out-turn now.

This will come into effect from: 01st March, 1999 under the reasonable formula.

Please take action accordingly.

You...<br>**Metadata:** {'original_doc_id': 121, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'REASONABLE PRICE FOR GREEN LEAF', 'issue_date': '23rd March 1999', 'reference_number': '28th June, 1994 (Circular No. MF / BL 136)'}<br>

---

**`Source Node 6/6`**

**Node ID:** b7585cce-7bc0-473c-ac87-876c06a1b546<br>**Similarity:** 2.647979259490967<br>**Text:** # TRANSLATIun

# Circuler Letter No-+iF / BJ, 62

Sri Lanka- Toa Board Division Sr

Tea Commissioner

P.0.Box 363, Colombo_1

30th August, 1984.

# To

All Registered Tea Manufacturers

# Dear Sir/Madam

# Payment of Reasonable Price for Green Leaf Tea Factories purchased by

Reference is requested to my Circular Letter No.iI/BL.60 of 6th February 1984, regarding the formula for calculating the reasonable price payable for green leaf.

The Government has decided to adopt the following measure...<br>**Metadata:** {'original_doc_id': 124, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Payment of Reasonable Price for Green Leaf Tea Factories purchased by', 'issue_date': '30th August, 1984', 'reference_number': 'Circular Letter No.II/BL.60 of 6th February 1984, and also Circular Letter No.I If / Bl. 62'}<br>

CPU times: user 1min 20s, sys: 8.52 s, total: 1min 29s
Wall time: 1min 30s


# Using Edgar's RagEvaluator
https://github.com/Edgar454/rag_evaluator?tab=readme-ov-file

Needs a GLHF key. Can be obtained from https://glhf.chat/

In [59]:
%%capture
!pip install -q git+https://github.com/edgar454/rag_evaluator.git

In [60]:
# # Configure GLHF from google secrets
from google.colab import userdata
os.environ["GLHF_API_KEY"]  = userdata.get('GLHF_API_KEY')
os.environ["GLHF_URL"] = userdata.get('GLHF_URL')

In [ ]:
# setting GLHF API Key
#os.environ["GLHF_API_KEY"] = (
#    os.environ["GLHF_API_KEY"]
#    if "GLHF_API_KEY" in os.environ
#    else getpass("GLHF_API_KEY: ")
#)

#os.environ["GLHF_URL"] = (
#    os.environ["GLHF_URL"]
#    if "GLHF_URL" in os.environ
#    else getpass("GLHF_URL Key: ")
#)

In [62]:
from rag_evaluator import RAGEvaluator
rageval = RAGEvaluator(query_engine=query_engine, api_key=os.environ["GLHF_API_KEY"] , mode = 'fast')

Initializing the evaluator


100%|██████████| 52/52 [00:00<00:00, 22688.42it/s]

Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' referenced before assignment
Error answering question: local variable 'result' re

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialization complete


In [ ]:
results = rageval.evaluate_rag()
print(results)

### Delete tables after use<br>
**Do not run** if tables need to be reused later.

In [ ]:
# table.drop()